In [1]:
import numpy as np
import xarray as xr

from src.simulator import simulate_trajectories

In [2]:
duacs_ds = xr.open_dataset("data/duacs_for_sim.nc")
era5_ds = xr.open_dataset("data/era5_for_sim.nc")
traj_ds = xr.open_dataset("data/traj_for_sim.nc")

In [3]:
x0 = np.asarray([traj_ds["lon"][0].values, traj_ds["lat"][0].values])

In [ ]:
sampled_params = np.array([0.3, 48, 0.3])
sampled_params = np.repeat(sampled_params[None, :], 10_000, axis=0)

In [7]:
sim_trajectories = simulate_trajectories(
    duacs_ds, era5_ds, x0, traj_ds["time"].values, sampled_params
)  # (N, T, 2)